In [4]:
from mmocr.apis import MMOCRInferencer
from ultralytics import YOLO
import time

In [7]:
from mmdeploy.backend.onnxruntime.init_plugins import get_ops_path
get_ops_path()

'/workspace/mmdeploy/mmdeploy/lib/libmmdeploy_onnxruntime_ops.so'

In [8]:
from mmdeploy.apis.utils import build_task_processor
from mmdeploy.utils import get_input_shape, load_config
import torch

deploy_cfg = '/workspace/mmdeploy/configs/mmocr/text-recognition/text-recognition_onnxruntime_static.py'
model_cfg = '/workspace/mmocr/configs/textrecog/abinet/abinet256x64.py'
device = 'cuda:0'
backend_model = ['../mmdeploy/models/mmocr/abinet/onnx/end2end.onnx']
image = './demo/resources/text_det.jpg'

# read deploy_cfg and model_cfg
deploy_cfg, model_cfg = load_config(deploy_cfg, model_cfg)

# build task and backend model
task_processor = build_task_processor(model_cfg, deploy_cfg, device)
model = task_processor.build_backend_model(backend_model)

# process input image
# input_shape = get_input_shape(deploy_cfg)
# model_inputs, _ = task_processor.create_input(image, input_shape)

# # do model inference
# with torch.no_grad():
#     result = model.test_step(model_inputs)

03/05 08:48:50 - mmengine - WARNING - Failed to search registry with scope "mmocr" in the "backend_text_recognizer" registry tree. As a workaround, the current "backend_text_recognizer" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmocr" is a correct scope, or whether the registry is initialized.
03/05 08:48:50 - mmengine - INFO - Successfully loaded onnxruntime custom ops from /workspace/mmdeploy/mmdeploy/lib/libmmdeploy_onnxruntime_ops.so


/workspace/mmocr/mmocr/models/textrecog/module_losses/ce_module_loss.py:101: UserWarning: padding does not exist in the dictionary
  warnings.warn(
/workspace/mmocr/mmocr/models/textrecog/postprocessors/base.py:60: UserWarning: padding does not exist in the dictionary
  warnings.warn(


In [5]:
# recog_config = "/workspace/mmocr/configs/textrecog/abinet/abinet_20e_st-an_mj.py"
# recog_ckpt = "work_dirs/abinet_20e_st-an_mj/epoch_3.pth"
recog_config = "/workspace/mmocr/configs/textrecog/abinet/abinet256x64.py"
recog_ckpt = "work_dirs/abinet256x64/epoch_3.pth"

# recog_config = "/workspace/mmocr/configs/textrecog/svtr/svtr-tiny_20e_st_mj.py"
# recog_ckpt = "work_dirs/svtr-tiny_20e_st_mj/epoch_20.pth"

In [63]:
import torch
def_config = "/workspace/mmocr/configs/textdet/dbnetpp/dbnetpp_resnet50-dcnv2_fpnc_1200e_icdar2015.py"
det_ckpt ="work_dirs/dbnetpp_resnet50-dcnv2_fpnc_1200e_icdar2015/epoch_20.pth"
device = "cuda:0"



# mod = torch.hub.load('ultralytics/yolov8', 'yolov8', '../detect_znak/data/models/Detector/yolov8/yolov8s-2023-02-11.pt', 'local', source='local')
mod = YOLO('../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx', task='detect')
# mod.ir_version = 11
# mod = YOLO("runs/detect/train3/weights/best.pt")
# Load models into memory
ocr =MMOCRInferencer(
#     det=def_config,
#     det_weights=det_ckpt,
    rec=recog_config,
    rec_weights=recog_ckpt,
     device=device,
)

Loads checkpoint by local backend from path: work_dirs/abinet256x64/epoch_3.pth


In [64]:
!rm -rf outputs/aug_rec/*
!rm -rf outputs/numberplate/*
!rm -rf outputs/car/*
ls = !ls  /workspace/datasets/ru_test/* #/workspace/datasets/ru_test/*
clock = time.time()
# for img in ls:
#     try:
i = 0        
for img in ls:
    results = mod(img, conf=0.8)
#         results[0].show()

    filename = img.split("/")[-1]
    results[0].save_crop(save_dir="./outputs", file_name=f"{filename.split('.')[0]}")# inference
#     print(results)
#     except:
#         continue
ls = !ls ./outputs/numberplate/*
for image in ls:
    input_shape = get_input_shape(deploy_cfg)
    model_inputs, _ = task_processor.create_input(image, input_shape)

    # do model inference
    with torch.no_grad():
        result = model.test_step(model_inputs)
    print(result[0].pred_text.item)

print(time.time() - clock)

Loading /workspace/test/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx for ONNX Runtime inference...


Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from /workspace/test/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx failed:/onnxruntime_src/onnxruntime/core/graph/model.cc:111 onnxruntime::Model::Model(onnx::ModelProto&&, const PathString&, const IOnnxRuntimeOpSchemaRegistryList*, const onnxruntime::logging::Logger&) Unknown model file format version.


In [55]:
mod.export(format='onnx', opset=11)

Ultralytics YOLOv8.1.23 🚀 Python-3.8.18 torch-1.13.1 CPU (Intel Xeon Gold 6248 2.50GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.4 MB)

ONNX: starting export with onnx 1.15.0 opset 11...
ONNX: export success ✅ 1.0s, saved as '../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx' (42.6 MB)

Export complete (3.1s)
Results saved to /workspace/nomerov/nomeroff-net/data/models/Detector/yolov8
Predict:         yolo predict task=detect model=../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx imgsz=640  
Validate:        yolo val task=detect model=../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx imgsz=640 data=/var/www/yolov8/data/npdata/numberplate_config.yaml  
Visualize:       https://netron.app


'../nomerov/nomeroff-net/data/models/Detector/yolov8/yolov8s-2023-02-11.onnx'

In [7]:
!rm -rf outputs/aug_rec/*
!rm -rf outputs/numberplate/*
!rm -rf outputs/car/*
ls = !ls  /workspace/datasets/ru_test/* #/workspace/datasets/ru_test/*
clock = time.time()
# for img in ls:
#     try:
i = 0        

results = mod(ls, conf=0.8)
#         results[0].show()
for img in ls:
    filename = img.split("/")[-1]
    results[i].save_crop(save_dir="./outputs", file_name=f"{filename[:-4]}")# inference
    i+=1
#     print(results)
#     except:
#         continue
ls = !ls ./outputs/numberplate/*
for image in ls:
    input_shape = get_input_shape(deploy_cfg)
    model_inputs, _ = task_processor.create_input(image, input_shape)

    # do model inference
    with torch.no_grad():
        result = model.test_step(model_inputs)
    print(result[0].pred_text.item)

print(time.time() - clock)

{'predictions': [{'rec_texts': ['M557AE154'],
   'rec_scores': [0.9988808499442207]}],
 'visualization': []}

In [9]:
results = mod("test_images/nomer6.png", save_crop=True) 


image 1/1 /workspace/mmocr/test_images/nomer6.png: 160x640 3 numberplates, 17.4ms
Speed: 1.5ms preprocess, 17.4ms inference, 1.0ms postprocess per image at shape (1, 3, 160, 640)
Results saved to runs/detect/predict


In [13]:
results[0].crop

AttributeError: 'Results' object has no attribute 'crop'. See valid attributes below.

    A class for storing and manipulating inference results.

    Attributes:
        orig_img (numpy.ndarray): Original image as a numpy array.
        orig_shape (tuple): Original image shape in (height, width) format.
        boxes (Boxes, optional): Object containing detection bounding boxes.
        masks (Masks, optional): Object containing detection masks.
        probs (Probs, optional): Object containing class probabilities for classification tasks.
        keypoints (Keypoints, optional): Object containing detected keypoints for each object.
        speed (dict): Dictionary of preprocess, inference, and postprocess speeds (ms/image).
        names (dict): Dictionary of class names.
        path (str): Path to the image file.

    Methods:
        update(boxes=None, masks=None, probs=None, obb=None): Updates object attributes with new detection results.
        cpu(): Returns a copy of the Re

In [45]:
results = mod.train(data="../data/data.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.1.17 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.16 🚀 Python-3.8.18 torch-1.10.1 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=runs/detect/train2/weights/best.pt, data=../data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False,

train: Scanning /workspace/data/train/labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /workspace/data/valid/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      4.34G     0.2314     0.1596     0.8087         16        640: 100%|██████████| 5/5 [00:00<00:00,  9.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 13.79it/s]

                   all         20         20      0.997       0.95      0.967      0.911



100 epochs completed in 0.066 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 22.5MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.16 🚀 Python-3.8.18 torch-1.10.1 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  8.82it/s]


                   all         20         20      0.997       0.95      0.966      0.945
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train3
